In [ ]:
# -----------------------------------------
# INSTALL REQUIRED LIBRARIES (Google Colab)
# -----------------------------------------
!pip install -q transformers datasets sentence-transformers hnswlib accelerate

# -----------------------------------------
# IMPORT LIBRARIES
# -----------------------------------------
from google.colab import drive, files
drive.mount('/content/drive')

import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
import hnswlib
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, pipeline
)
import torch
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv("ARS_DATA.csv")

# -----------------------------------------
# STEP 2: CLEAN AND ENCODE DATA
# -----------------------------------------
def clean_and_encode_data(df):
    df.dropna(inplace=True)
    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]
        df[col] = encoders[col].fit_transform(df[col])
    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])
    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)

# -----------------------------------------
# STEP 3: CREATE HUMAN-READABLE DESCRIPTIONS
# -----------------------------------------
def create_description(df):
    df['Full_Description'] = df.apply(lambda row:
        f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
        f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
        f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
        axis=1)
    return df

df = create_description(df)

# -----------------------------------------
# STEP 4: GENERATE EMBEDDINGS
# -----------------------------------------
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

# -----------------------------------------
# STEP 5: BUILD HNSW INDEX
# -----------------------------------------
def build_hnsw_index(embeddings, ef=100, M=16):
    dim = embeddings.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=ef, M=M)
    index.add_items(embeddings, ids=list(range(len(embeddings))))
    index.set_ef(ef)
    return index

index = build_hnsw_index(embeddings)

# -----------------------------------------
# STEP 6: PREPARE DATA FOR GPT FINE-TUNING
# -----------------------------------------
def prepare_finetuning_data(df):
    df['input'] = df['Full_Description'].apply(lambda x: f"{x} => Recommended meal and workout:")
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

ds = prepare_finetuning_data(df)

# -----------------------------------------
# STEP 7–9: TOKENIZATION AND TRAINING
# -----------------------------------------
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = ds.map(tokenize_function, batched=True)
small_ds = tokenized_dataset.select(range(min(5000, len(tokenized_dataset))))

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/distilgpt2_finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir='/content/drive/MyDrive/logs',
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# -----------------------------------------
# STEP 10: SAVE EVERYTHING
# -----------------------------------------
trainer.save_model("/content/drive/MyDrive/RAG_distilgpt2_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/RAG_distilgpt2_finetuned")
df.to_csv("/content/drive/MyDrive/RAG_ARS_df.csv", index=False)
np.save("/content/drive/MyDrive/RAGARS_embeddings.npy", embeddings)
index.save_index("/content/drive/MyDrive/RAGARS_hnsw_index.bin")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving ARS_DATA.csv to ARS_DATA.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

<ipython-input-3-066b792e1fc1>:114: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [ ]:
!pip install -q transformers sentence-transformers hnswlib

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load saved artifacts
df = pd.read_csv("/content/drive/MyDrive/ARS_df.csv")
embeddings = np.load("/content/drive/MyDrive/ARS_embeddings.npy")

dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index("/content/drive/MyDrive/ARS_hnsw_index.bin")
index.set_ef(100)

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/distilgpt2_finetuned")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/RAG_distilgpt2_finetuned").to("cuda")
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
def recommend_with_rag(query, embedder, index, df, gen_pipeline, top_k=5):
    # Step 1: Embed the user query
    query_vec = embedder.encode([query])

    # Step 2: Retrieve top-k similar profiles using HNSW
    ids, _ = index.knn_query(query_vec, k=top_k)
    retrieved_contexts = df.iloc[ids[0]][['Full_Description', 'Meal Name_original', 'Meal Time']].values.tolist()

    # Step 3: Construct contextual knowledge as RAG input
    context_blocks = []
    for desc, meal, time in retrieved_contexts:
        context_blocks.append(f"{desc} => Meal: {meal} at {time}")

    rag_context = "\n".join(context_blocks)

    # Step 4: Construct prompt for LLM
    prompt = f"""You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

{rag_context}

Now, for the following user:
{query}
Provide a recommended meal and workout plan:"""

    # Step 5: Generate using LLM
    response = gen_pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.9)[0]['generated_text']
    return response


query = "Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5"
print(recommend_with_rag(query, embedder, index, df, gen_pipeline))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda:0


You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

Female, 60 years old, -1.56 cm, 1.16 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.22 => Meal: Quinoa Bowl at 2
Female, 40 years old, 1.23 cm, 1.42 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 0.25 => Meal: Quinoa Bowl at 1
Female, 48 years old, -1.35 cm, 1.21 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.05 => Meal: Tofu Salad at 0
Female, 27 years old, -1.63 cm, 0.26 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 1.35 => Meal: Grilled Chicken at 0
Female, 22 years old, 1.50 cm, -0.79 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI -1.27 => Meal: Beef Stir Fry at 3

Now, for the following user:
Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5
Provide a recommended meal and workout plan: Paneer Stir Fry, Sedentary exercise, BMI 2.47 => Recommended meal and workout: Keto Omelette Tofu Salad Bow

In [ ]:
query = "Female, 35 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5"
print(recommend_with_rag(query, embedder, index, df, gen_pipeline))


You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

Female, 48 years old, -1.35 cm, 1.21 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.05 => Meal: Tofu Salad at 0
Female, 60 years old, -1.56 cm, 1.16 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.22 => Meal: Quinoa Bowl at 2
Female, 35 years old, 0.48 cm, -1.32 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI -1.28 => Meal: Quinoa Bowl at 1
Female, 54 years old, 1.64 cm, 2.00 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 0.38 => Meal: Chickpea Curry at 3
Female, 27 years old, -1.63 cm, 0.26 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 1.35 => Meal: Grilled Chicken at 0

Now, for the following user:
Female, 35 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5
Provide a recommended meal and workout plan: Salmon Fillet Tofu Salad (Keto Omelette) Stir Fryer Stir Fryer Stir Fryer Keto Omelette Stir Fryer Keto Om